In [74]:
import PIL
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image,ImageOps,ImageFilter
import keras
from keras import layers
import os
from tensorflow.keras.utils import Sequence
from tensorflow.keras import Sequential
import tensorflow as tf
from keras.preprocessing import image  
import cv2
import pickle

In [12]:
from keras.models import load_model
model = load_model('model\self_trained\distracted-23-1.00.hdf5')

In [5]:
image = PIL.Image.open("car_test_1.jpg")
image = image.resize((64,64))

In [13]:
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(128, 128))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [14]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True 

test_tensor = path_to_tensor("car_test_1.jpg").astype('float32')/255 - 0.5

In [15]:
y_pred = model.predict_classes(test_tensor)

In [16]:
ypred_class = np.argmax(y_pred)
print(ypred_class)

0


In [26]:
infile = open("pickle_files\labels_list.pkl",'rb')
new_dict = pickle.load(infile)
infile.close()
print(new_dict)

{'c3': 0, 'c9': 1, 'c2': 2, 'c4': 3, 'c6': 4, 'c1': 5, 'c8': 6, 'c5': 7, 'c0': 8, 'c7': 9}


In [75]:
from keras.models import load_model
vgg_model = load_model('model/vgg16/distracted-168-0.92.hdf5')

In [86]:
def path_to_tensor_vgg(img_path):
    # loads RGB image as PIL.Image.Image type
    #img = image.load_img(img_path)
    img = cv2.imread(img_path)
    img = cv2.resize(img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    # convert PIL.Image.Image type to 3D tensor with shape (128, 128, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 128, 128, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [129]:
infile = open("pickle\labels_list_vgg16.pkl",'rb')
new_dict = pickle.load(infile)
infile.close()
print(new_dict)

{'c3': 0, 'c4': 1, 'c1': 2, 'c8': 3, 'c2': 4, 'c6': 5, 'c7': 6, 'c0': 7, 'c9': 8, 'c5': 9}


In [130]:
ImageFile.LOAD_TRUNCATED_IMAGES = True                 
vgg_test_tensor = path_to_tensor_vgg('car_test_8.jpg').astype('float32')/255 - 0.5

In [131]:
ypred_test = model.predict(vgg_test_tensor,verbose=1)
ypred_class = np.argmax(ypred_test,axis=1)

1/1 [==============================] - 0s 68ms/step


In [132]:
print(ypred_class)

[4]
